# SAHI Benchmark

## 1. FULL INFERENCE

In [29]:
# arrange an instance segmentation model for test
from sahi.utils.yolov5 import (
    download_yolov5s6_model,
)

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.scripts.coco_error_analysis import analyse
from sahi.scripts.coco_evaluation import evaluate
from IPython.display import Image
from pathlib import Path
import json
import os
from pathlib import Path
from PIL import Image

In [30]:
source_image_dir = "../resources/FLL_VAL/images/"
source_label_dir = "../resources/FLL_VAL/labels/"

coco_m_path = '../resources/models/yolov5m.pt'
fll_model_221007_path = '../resources/models/221007/best.pt'
fll_model_221012_path = '../resources/models/221012/best.pt'

In [31]:
fll_model_221012 = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=fll_model_221012_path,
    confidence_threshold=0.25,
    device="cuda:0"
)

In [32]:
model = fll_model_221012
model_path = fll_model_221012_path

In [43]:
def initial_extract(img_dir, label_dir, out_dir):
    if os.path.exists(os.path.join(out_dir, 'val.json')):
        os.remove(os.path.join(out_dir, 'val.json'))
    
    licenses = [
        {
            "name": "",
            "id": 0,
            "url": ""
        }
    ]

    info_ = [
        {
            "contributor": "",
            "date_created": "",
            "description": "",
            "url": "",
            "version": "",
            "year": ""
        }
    ]

    categories = [
        {
            "id": 0,
            "name": "Buoy",
            "supercategory": ""
        },
        {
            "id": 1,
            "name": "Boat",
            "supercategory": ""
        },
        {
            "id": 2,
            "name": "Channel Marker",
            "supercategory": ""
        },
        {
            "id": 3,
            "name": "Speed Warning Sign",
            "supercategory": ""
        }
    ]

    img_idx = 0
    annot_idx = 0

    imgs_list = []
    annots_list = []

    for label_file in sorted(os.listdir(label_dir)):
        label_file_ = os.path.join(label_dir, label_file)
        img_file_ = os.path.join(img_dir, f'{os.path.splitext(label_file)[0]}.jpg')
        img = Image.open(img_file_)
        image_w, image_h = img.size

        imgs_list.append({
            'id': img_idx,
            'width': image_w,
            'height': image_h,
            'file_name': f'{os.path.splitext(label_file)[0]}.jpg',
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        with open(label_file_, 'r') as label_f:
            labels = label_f.readlines()

            for label in labels:
                cat, xc, yc, label_normalized_w, label_normalized_h = list(map(lambda x: int(x) if len(x) == 1 else float(x), label.split()))
                label_w, label_h = image_w * label_normalized_w, image_h * label_normalized_h
                xmin, ymin = (image_w * xc) - (label_w / 2), (image_h * yc) - (label_h / 2)
                
                xmin = 0 if xmin < 0 else xmin
                ymin = 0 if ymin < 0 else ymin

                annots_list.append({
                    'id': annot_idx,
                    'image_id': img_idx,
                    'category_id': cat,
                    'area': int(label_h * label_w),
                    'bbox': [
                        xmin,
                        ymin,
                        label_w,
                        label_h
                    ],
                    'iscrowd': 0,
                    'attributes': {
                        'type': '',
                        'occluded': False
                    },
                    'segmentation': []
                })

                annot_idx += 1

        img_idx += 1

    out_dict = {
        'licenses': licenses,
        'info': info_,
        'categories': categories,
        'images': imgs_list,
        'annotations': annots_list
    }
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    
    with open(os.path.join(out_dir, 'val.json'), 'w') as out_f:
        print(os.path.join(out_dir, 'val.json'))
        json.dump(out_dict, out_f)
        
    return os.path.join(out_dir, 'val.json')

In [44]:
# initial_extract(img_dir, label_dir, out_dir)
gt_json_path = initial_extract(source_image_dir, source_label_dir, str(Path(source_image_dir).parent))

../resources/FLL_VAL/val.json


In [58]:
MODEL_TYPE = "yolov5"
MODEL_PATH = model_path
MODEL_CONFIG_PATH = ""
EVAL_IMAGES_FOLDER_DIR = source_image_dir
EVAL_DATASET_JSON_PATH = gt_json_path
INFERENCE_SETTING = "AVIKUS_FL"
EXPORT_VISUAL = False
MAX_DETECTIONS = 500

In [59]:
INFERENCE_SETTING_TO_PARAMS = {
    "AVIKUS_FL": {
        "no_standard_prediction": False,
        "no_sliced_prediction": True,
        "slice_size": 512,
        "overlap_ratio": 0.15,
        "match_threshold": 0.5,
        "postprocess_class_agnostic": False,
        "single_row_y_start": 200,
    },
}

setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

In [61]:
result = predict(
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.25,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    image_size=640,
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="GREEDYNMM",
    postprocess_match_metric="IOS",
#     postprocess_type="NMS",
#     postprocess_match_metric="IOU",
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/mAP_TEST",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=True,
    single_row_predict=True,
    single_row_y_start=setting_params["single_row_y_start"]
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

indexing coco dataset annotations...


Performing inference on images:   0%|                                | 6/5906 [00:00<03:23, 28.95it/s]

Prediction time is: 12.84 ms
Prediction time is: 13.51 ms
Prediction time is: 13.66 ms
Prediction time is: 12.27 ms
Prediction time is: 13.06 ms
Prediction time is: 13.12 ms


Performing inference on images:   0%|                               | 12/5906 [00:00<03:20, 29.46it/s]

Prediction time is: 13.04 ms
Prediction time is: 12.29 ms
Prediction time is: 13.01 ms
Prediction time is: 12.94 ms
Prediction time is: 12.52 ms
Prediction time is: 12.54 ms


...

Prediction time is: 14.29 ms
Prediction time is: 13.23 ms
Prediction time is: 13.33 ms
Prediction time is: 13.17 ms
Prediction time is: 13.22 ms
Prediction time is: 13.86 ms
Prediction time is: 13.51 ms


Performing inference on images: 100%|████████████████████████████▉| 5903/5906 [03:12<00:00, 30.54it/s]

Prediction time is: 13.36 ms
Prediction time is: 13.70 ms
Prediction time is: 13.13 ms
Prediction time is: 13.55 ms
Prediction time is: 13.27 ms
Prediction time is: 13.42 ms
Prediction time is: 13.76 ms


Performing inference on images: 100%|█████████████████████████████| 5906/5906 [03:12<00:00, 30.60it/s]


Prediction time is: 12.48 ms
Prediction results are successfully exported to runs/mAP_TEST/AVIKUS_FL4
Model loaded in 0.08326005935668945 seconds.
Slicing performed in 0 seconds.
Prediction performed in 76.01503992080688 seconds.


In [63]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.32s).
Accumulating evaluation results...
DONE (t=0.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.113
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.244
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.114
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.453
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.492
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.037
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

## DSL

In [70]:
!pwd

/home/yyj-avikus/sahi/demo


In [71]:
dsl_result_json = "../../aiboat/aiboat/APP/NAS/deepstream-services-library/examples/python/result_standard_pred.json"

In [72]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=dsl_result_json,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.44s).
Accumulating evaluation results...
DONE (t=0.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.236
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.102
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.460
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.465
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

## 2. SLICE INFERENCE

### Slice 좌표 찾기

In [75]:
source_image_dir = "../resources/FLL_VAL/images/"
image_files = sorted([fn for fn in os.listdir(source_image_dir) if fn.endswith("jpg")])

In [82]:
from typing import List
import copy
import matplotlib.pyplot as plt
import cv2
from ipywidgets import interact
%matplotlib inline 
from sahi import slicing
from sahi.slicing import slice_image

slicing.logger.setLevel(slicing.logging.INFO)

# single_row_y_start: int = 200,
@interact(index=(0, len(image_files)-1), slice_size=(0, 720), overlap_ratio=(0, 0.5, 0.05), single_row_y_start=(0, 512))
def visualize_slice_rect(index=0, slice_size=512, overlap_ratio=0.2, single_row_y_start=200):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    
    res = slice_image(image_path, 
                      slice_width=slice_size,
                      slice_height=slice_size,
                      overlap_height_ratio=overlap_ratio,
                      overlap_width_ratio=overlap_ratio,
                      single_row_y_start=single_row_y_start,
                      single_row_predict=True,
                      verbose=1)

    image = cv2.imread(image_path)
    image = copy.deepcopy(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    for start_pixel in res.starting_pixels:
        print(start_pixel)
        cv2.rectangle(image,
                      start_pixel,
                      [s1+s2 for s1, s2 in zip(start_pixel,[slice_size,slice_size])],
                      color=(255, 0, 0),
                      thickness=2)

    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=5905), IntSlider(value=512, description='sli…

### SAHI

In [83]:
MODEL_TYPE = "yolov5"
MODEL_PATH = model_path
MODEL_CONFIG_PATH = ""
EVAL_IMAGES_FOLDER_DIR = source_image_dir
EVAL_DATASET_JSON_PATH = gt_json_path
INFERENCE_SETTING = "AVIKUS_FL"
EXPORT_VISUAL = False
MAX_DETECTIONS = 500

In [84]:
INFERENCE_SETTING_TO_PARAMS = {
    "AVIKUS_FL": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 640,
        "overlap_ratio": 0.25,
        "match_threshold": 0.5,
        "postprocess_class_agnostic": False,
        "single_row_y_start": 200,
    },
}

setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

In [85]:
result = predict(
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
#     model_confidence_threshold=,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
#     image_size=,
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="GREEDYNMM",
    postprocess_match_metric="IOS",
#     postprocess_type="NMS",
#     postprocess_match_metric="IOU",
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/mAP_TEST",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=True,
    single_row_predict=True,
    single_row_y_start=setting_params["single_row_y_start"]
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

indexing coco dataset annotations...


Performing inference on images:   0%|                                | 1/5906 [00:00<12:33,  7.84it/s]

Performing prediction on 4 number of slices.
Prediction time is: 107.75 ms
Performing prediction on 4 number of slices.


Performing inference on images:   0%|                                | 3/5906 [00:00<12:58,  7.58it/s]

Prediction time is: 109.41 ms
Performing prediction on 4 number of slices.
Prediction time is: 115.05 ms
Performing prediction on 4 number of slices.


...

Prediction time is: 107.03 ms
Performing prediction on 4 number of slices.
Prediction time is: 102.44 ms
Performing prediction on 4 number of slices.


Performing inference on images: 100%|████████████████████████████▉| 5905/5906 [12:12<00:00,  8.23it/s]

Prediction time is: 102.06 ms
Performing prediction on 4 number of slices.
Prediction time is: 99.44 ms
Performing prediction on 4 number of slices.


Performing inference on images: 100%|█████████████████████████████| 5906/5906 [12:13<00:00,  8.06it/s]


Prediction time is: 101.69 ms
Prediction results are successfully exported to runs/mAP_TEST/AVIKUS_FL5
Model loaded in 0.07593131065368652 seconds.
Slicing performed in 9.363306999206543 seconds.
Prediction performed in 613.5637776851654 seconds.


In [86]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.80s).
Accumulating evaluation results...
DONE (t=0.22s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.263
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.325
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.183
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

### DSL

In [ ]:
# network-input-shape= 5;3;640;640
# roi-params-src-0=0;0;1920;1080;0;200;640;640;480;200;640;640;960;200;640;640;1280;200;640;640

In [89]:
dsl_result_json = "../../aiboat/aiboat/APP/NAS/deepstream-services-library/examples/python/result_slice_pred.json"

In [90]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=dsl_result_json,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.07s).
Accumulating evaluation results...
DONE (t=0.28s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.106
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.256
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.286
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50:0.95 | a